# Deploy de Modelos de Machine Learning

## Lab - Deploy na Prática - App Web Para Previsão de Preço de Produtos

### Instalando e Carregando Pacotes

In [95]:
import pandas as pd
import numpy as np

# Número de amostras
num_samples = 1000

# Semente para reprodução
np.random.seed(42)

# Gerando os dados
distancia_km = np.random.uniform(1, 20, num_samples)  # de 1 a 200 km
horario_pedido = np.random.randint(0, 24, num_samples)  # 0 a 23h
valor_pedido = np.random.uniform(10, 200, num_samples)  # R$10 a R$200
pedidos_simultaneos = np.random.randint(1, 20, num_samples)  # 1 a 20 pedidos
clima = np.random.randint(0, 11, num_samples)  # Severidade do clima 0 a 10
congestionamento = np.random.randint(0, 11, num_samples)  # Severidade do congestionamento 0 a 10

# Taxa de entrega com peso maior para distância
taxa_entrega = (
    distancia_km * 2.5 +               # distância manda demais
    pedidos_simultaneos * 1.2 +        # impacto direto na carga do motoboy
    congestionamento * 0.8 +           # engarrafamento pesa
    clima * 0.5 +                      # chuva e tals
    horario_pedido * 0.1 +             # madrugada/horário de pico
    valor_pedido * 0.005 +             # quase irrelevante
    np.random.normal(0, 1, num_samples)  # ruído natural
)

# Criando DataFrame
df = pd.DataFrame({
    'distancia_km': distancia_km,
    'horario_pedido': horario_pedido,
    'valor_pedido': valor_pedido,
    'pedidos_simultaneos': pedidos_simultaneos,
    'clima': clima,
    'congestionamento': congestionamento,
    'taxa_entrega': taxa_entrega
})

# Exporta como CSV
df.to_csv('dados/dataset_taxa_entrega_realista.csv', index=False)


In [96]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.8.10


In [98]:
# Imports
import joblib
import sklearn
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

## Carregando e Explorando os Dados

In [115]:
# Carrega os dados
df = pd.read_csv('dados/dataset_taxa_entrega_realista.csv')

In [116]:
# Visualiza os dados
df.head()

,distancia_km,horario_pedido,valor_pedido,pedidos_simultaneos,clima,congestionamento,taxa_entrega
0,8.116262,14,107.979091,1,7,9,34.445031
1,19.063572,11,184.684475,4,6,3,61.720453
2,14.907885,15,104.423062,13,7,0,57.224293
3,12.374511,23,198.510023,13,10,6,60.149381
4,3.964354,18,171.770742,13,7,1,33.099888


In [113]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   distancia_km         1000 non-null   float64
 1   horario_pedido       1000 non-null   int64  
 2   valor_pedido         1000 non-null   float64
 3   pedidos_simultaneos  1000 non-null   int64  
 4   clima                1000 non-null   int64  
 5   congestionamento     1000 non-null   int64  
 6   taxa_entrega         1000 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 54.8 KB


In [114]:
# Shape
df.shape

(1000, 7)

In [117]:
# Colunas
df.columns

Index(['distancia_km', 'horario_pedido', 'valor_pedido', 'pedidos_simultaneos',
       'clima', 'congestionamento', 'taxa_entrega'],
      dtype='object')

### Construindo o Modelo

In [118]:
# Separa X e y
X = df.drop('taxa_entrega', axis=1)
y = df['taxa_entrega']

In [119]:
# Divide em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=0)

In [120]:
# Cria o modelo
modelo = GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    random_state=0
)

In [121]:
# Treina o modelo
modelo.fit(X_treino, y_treino)

GradientBoostingRegressor(learning_rate=0.05, max_depth=7, n_estimators=1000,
                          random_state=0, subsample=0.8)

In [122]:
# Faz previsões
previsoes = modelo.predict(X_teste)

In [123]:
# Avalia
print("R2 Score:", r2_score(y_teste, previsoes) * 100)

R2 Score: 98.52756490246065


In [124]:
# Salva o modelo em disco
joblib.dump(modelo, '../modelo/modelo.pkl')

['../modelo/modelo.pkl']